In [2]:
# Step 0: 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 작업 디렉토리 설정
import os
project_path = '/content/drive/MyDrive/aladin-project'
%cd {project_path}
print(f"현재 작업 디렉토리: {os.getcwd()}")

# Step 1: 라이브러리 import
import pandas as pd
import numpy as np

# Step 2: 파일 불러오기
try:
    df_aladin = pd.read_csv('data/raw/aladin.csv', index_col=0, dtype={'item_id': str})
    print(" aladin.csv 로드 완료")
    df_aladin['category'] = df_aladin['category'].astype(object)
except FileNotFoundError:
    print(" 오류: aladin.csv 파일을 찾을 수 없습니다.")
    raise

try:
    df_category = pd.read_csv('data/raw/category_mapping.csv', dtype={'item_id': str})
    print(" category_mapping.csv 로드 완료")
except FileNotFoundError:
    print(" 오류: category_mapping.csv 파일을 찾을 수 없습니다.")
    raise

print("=" * 50)

# Step 3: 데이터 병합 및 카테고리 업데이트
df_merged = pd.merge(
    df_aladin,
    df_category.drop_duplicates(subset=['item_id']),
    on='item_id',
    how='left'
)

has_new_category = df_merged['real_category'].notnull() & (df_merged['real_category'] != 'N/A')
print(f" {has_new_category.sum()}개 행의 카테고리를 업데이트합니다.")

df_merged.loc[has_new_category, 'category'] = df_merged.loc[has_new_category, 'real_category']
df_merged = df_merged.drop(columns=['real_category'])
print(" 카테고리 업데이트 완료")
print("=" * 50)

# Step 4: 카테고리 없는 행 삭제
condition_to_drop = (df_merged['category'].isnull()) | (df_merged['category'] == 'N/A')
num_dropped = condition_to_drop.sum()
df_cleaned = df_merged[~condition_to_drop]

print(f" 카테고리가 비어있거나 'N/A'인 {num_dropped}개 행 삭제")
print(f" 최종 데이터: {len(df_cleaned)}개 행")
print("=" * 50)

# Step 5: 최종 파일 저장
os.makedirs('data/processed', exist_ok=True)
save_path = 'data/processed/aladin_final_cleaned.csv'
df_cleaned.to_csv(save_path, encoding='utf-8-sig', index=True)
print(f" 최종 파일 저장 완료: {save_path}")

# Step 6: 최종 데이터 확인
print("\n 최종 데이터 요약")
print(f"- 총 행 수: {len(df_cleaned)}")
print(f"- 고유 도서: {df_cleaned['item_id'].nunique()}개")
print(f"- 카테고리: {df_cleaned['category'].nunique()}개")
print(f"\n데이터 샘플:")
df_cleaned.head(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/aladin-project
현재 작업 디렉토리: /content/drive/MyDrive/aladin-project
 aladin.csv 로드 완료
 category_mapping.csv 로드 완료
 3426개 행의 카테고리를 업데이트합니다.
 카테고리 업데이트 완료
 카테고리가 비어있거나 'N/A'인 19개 행 삭제
 최종 데이터: 3426개 행
 최종 파일 저장 완료: data/processed/aladin_final_cleaned.csv

 최종 데이터 요약
- 총 행 수: 3426
- 고유 도서: 1890개
- 카테고리: 22개

데이터 샘플:


,year,month,rank,category,title,price,star_score,item_id
0,2020,1,1,어린이,흔한남매 3,10800,9.9,223274669
1,2020,1,2,인문학,지적 대화를 위한 넓고 얕은 지식 : 제로 편,20700,8.5,219841523
3,2020,1,4,어린이,추리 천재 엉덩이 탐정 9,10800,9.8,227129549
4,2020,1,5,자기계발,에이트,17100,6.4,212657645
5,2020,1,6,만화,어느 날 공주가 되어버렸다 3 (종이 인형 책(128p) + 투명스티커 4종 + 안...,32400,9.4,227693700
6,2020,1,7,어린이,117층 나무 집,12600,9.8,222102869
7,2020,1,8,에세이,스스로 행복하라,10800,7.4,223726675
8,2020,1,9,어린이,설민석의 세계사 대모험 4,10800,9.6,226928892
9,2020,1,10,자기계발,나는 무조건 합격하는 공부만 한다,13500,8.3,220642182
10,2020,1,11,소설/시/희곡,너와 함께라면 인생도 여행이다,13500,9.5,221601434
